In [52]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, Span
from bokeh.io import curdoc
from bokeh.themes import built_in_themes, Theme

from bokeh.layouts import row
import time
import pytz
import zoneinfo
from pytz import timezone

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
#filter on user data only
base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
# if current_user.id==2:
df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)
# else:
#     df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(current_user.id),db.session.bind)

In [3]:
#This stays exactly
colNames=[i[len('health_description_'):] for i in list(df_health_description.columns)]
col_names_dict={i:j for i,j in zip(list(df_health_description.columns),colNames)}
df_health_description.rename(columns=col_names_dict, inplace=True)

In [4]:
def plot_text_format(x):
    return ('%.1f' % x).rstrip('0').rstrip('.')

In [44]:
#This stays exactly
df1=df_health_description.loc[df_health_description.metric1_carido<100]
df1=df1.sort_values(by=['datetime_of_activity'])
obs_x1=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df1.datetime_of_activity]
obs_y1=df1.metric1_carido
obs_y1_formatted=[ plot_text_format(i) for i in obs_y1]

date_end=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[-1],'%Y-%m-%d %H:%M:%S.%f')
date_end=date_end+ timedelta(days=5)
date_start=date_end- timedelta(days=7)

In [45]:
#get x and y's for activity
df2=df_health_description.loc[(df_health_description.var_type=='Activity') ]
#1 time (obs_x)
obs_x2=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df2.datetime_of_activity]
#3 activity (obs_y2)
obs_y2=df2.var_activity.to_list()

In [75]:
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date_start,date_end),y_range=(-10,90),width=800, height=300)

#add cardio_metric1
source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1_formatted))
glyph1 = Text(text="text",text_font_size={'value': '10px'})
fig1.add_glyph(source1, glyph1)

for a,b in zip(obs_x2,obs_y2):
    #add activity data
    source2 = ColumnDataSource(dict(x=[a], y=[80], text=[b]))
    glyph2 = Text(text="text", text_color="#414444", text_font_size={'value': '10px'},
                 x_offset=-10,angle=-1.58)
    
    #add line for activity data
    line_start_time=time.mktime(a.timetuple())*1000
    important_time = Span(location=line_start_time, dimension='height', line_color="#414444", line_dash='dashed', line_width=1)
    fig1.add_glyph(source2, glyph2)
    fig1.add_layout(important_time)

fig1.add_glyph(source2, glyph2)
fig1.ygrid.grid_line_color = None
fig1.yaxis.major_label_text_color = None
fig1.yaxis.major_tick_line_color = None
fig1.yaxis.minor_tick_line_color = None

curdoc().theme = Theme(filename=r"D:\OneDrive\Documents\professional\20210710lifeBuddy\chart_theme_2.yml")
show(fig1)

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\bokeh\models\plots.py:766: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
